In [ ]:
    def forward(
        self,
        tgt: Tensor,
        memory: Tensor,
        tgt_mask: Optional[Tensor] = None,
        memory_mask: Optional[Tensor] = None,
        tgt_key_padding_mask: Optional[Tensor] = None,
        memory_key_padding_mask: Optional[Tensor] = None,
        tgt_is_causal: bool = False,
        memory_is_causal: bool = False,
    ) -> Tensor:

        x = tgt
        if self.norm_first:
            x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask, tgt_is_causal)
            x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask, memory_is_causal)
            x = x + self._ff_block(self.norm3(x))
        else:
            x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask, tgt_is_causal))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask, memory_is_causal))
            x = self.norm3(x + self._ff_block(x))

        return x

* key_padding_mask: Trueが無視される。つまり自分の実装は正しい。
* attn_mask: 同じく、Trueは無視される。
    * 一番最後の行がなぜいるか分からない。全てのターゲットトークン(EOS含め)が見えるのはどういう状況？

In [3]:
import torch

sz = 5
torch.triu(torch.ones((sz, sz), dtype=bool), diagonal=1)

tensor([[False,  True,  True,  True,  True],
        [False, False,  True,  True,  True],
        [False, False, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [3]:
import torch
from torch import nn

In [6]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
input, m(input)

(tensor([[ 1.1366,  0.7714, -0.4227],
         [ 1.5066, -0.7554, -0.3128]]),
 tensor([[0.5251, 0.3645, 0.1104],
         [0.7897, 0.0822, 0.1280]]))